# Using ROBOKOP's enrichment service

ROBOKOP's enrichment service, takes a list of identifiers, and returns a list of identifiers (of a given type) that are enriched for connections to the original set.  The calculation here is well-known as "GO Enrichment" in which given a gene list, the enrichment finds GO terms that are associated with the genes in the set much more frequently than chance would suppose.

The following function shows how the enrichment service can be called.  Note that enrichment takes a POST request, so that a list of identifiers can be easily passed.

In [1]:
robokop_server = 'robokop.renci.org'

In [2]:
import requests
import json
import pandas as pd

In [3]:
def enrichment(type1,identlist,type2,threshhold=None,maxresults=None,numtype1=None,include_descendants=None,rebuild=None):
    url=f'http://{robokop_server}/api/simple/enriched/{type1}/{type2}'
    params = { 'threshhold': threshhold, 'max_results': maxresults, 
              'num_type1':numtype1, 'identifiers': identlist, 
              'include_descendants':include_descendants, 'rebuild': rebuild }
    params = { k:v for k,v in params.items() if v is not None }
    response=requests.post(url, json = params)
    print( f'Return Status: {response.status_code}' )
    if response.status_code == 200:
        return response.json()
    return []

## Basic Usage

Let's find phenotypes that are enriched for diabetes.  In particular, let's choose two versions of diabetes, type 1 diabetes mellitus (MONDO:0005147) and type two diabetes mellitus (MONDO:0005148).   The return is a list of dicts, which is easily converted to a pandas dataframe.

In [5]:
diabetes = ['MONDO:0005147','MONDO:0005148']
type1='disease'
type2='phenotypic_feature'

In [5]:
values = enrichment(type1, diabetes, type2)
diabetes_enriched_phenotypes = pd.DataFrame(values)
diabetes_enriched_phenotypes

Return Status: 200


,id,name,p
0,NCIT:C76325,Birth Weight,0.000001
1,NCIT:C81328,Body Weight,0.000004
2,HP:0005974,Episodic ketoacidosis,0.000020
3,HP:0011016,Abnormality of urine glucose concentration,0.000021
4,HP:0006279,Beta-cell dysfunction,0.000024
5,HP:0030082,Abnormal drinking behavior,0.000026
6,HP:0005979,Metabolic ketoacidosis,0.000035
7,HP:0100739,Bulimia,0.000035
8,HP:0012093,Abnormality of endocrine pancreas physiology,0.000036
9,HP:0006476,Abnormality of the pancreatic islet cells,0.000037


## Limiting results with maxresults and threshhold

By default, the enrichment service returns the top 100 results.   That number can be controlled with the maxresults parameter.  If set to 0, all results are returned

In [6]:
top_100 = enrichment(type1, diabetes, type2)
all_results = enrichment(type1, diabetes, type2, maxresults = 0)
top_10 = enrichment(type1, diabetes, type2, maxresults = 10)

print(f'Calling with no maxresults parameter returned {len(top_100)} results')
print(f'Calling with maxresults=0 parameter returned {len(all_results)} results')
print(f'Calling with maxresults=10 parameter returned {len(top_10)} results')

Return Status: 200
Return Status: 200
Return Status: 200
Calling with no maxresults parameter returned 100 results
Calling with maxresults=0 parameter returned 250 results
Calling with maxresults=10 parameter returned 10 results


A node's score in an enrichment calculation is a p-value, in this case calculated from a hypergeometric distribution.  Because it's a p-value, lower is better.  The default (maximum) p-value returned is 0.05.  Note that this p-value is uncorrected for multiplicity at the moment.  The maximum p-value threshold can be specified to control the amount of results that are returned:

In [7]:
p1m4 = enrichment(type1, diabetes, type2, threshhold=0.0001)
print (f'{len(p1m4)} results returned with p < 0.0001')
pd.DataFrame(p1m4)

Return Status: 200
18 results returned with p < 0.0001


,id,name,p
0,NCIT:C76325,Birth Weight,0.000001
1,NCIT:C81328,Body Weight,0.000004
2,HP:0005974,Episodic ketoacidosis,0.000020
3,HP:0011016,Abnormality of urine glucose concentration,0.000021
4,HP:0006279,Beta-cell dysfunction,0.000024
5,HP:0030082,Abnormal drinking behavior,0.000026
6,HP:0005979,Metabolic ketoacidosis,0.000035
7,HP:0100739,Bulimia,0.000035
8,HP:0012093,Abnormality of endocrine pancreas physiology,0.000036
9,HP:0006476,Abnormality of the pancreatic islet cells,0.000037


## Caching and rebuilding

ROBOKOP maintains cached results.  The cache is built both opportunistically (including the results of all previous queries) and proactively (pre-loading data that expected to be heavily used).  By default, expand only looks in its cache.  If a result has not been previously cached, then this call will not return anything (and may return a status code of 500).

If a user wants to force the service to look beyond its local cache, it sends a parameter `rebuild=True`, as seen in the NPC1 examples above.

If a user wants to be sure to retreive all relevant data, they should use `rebuild=True`, but this will be at the expense of performance.  In order to increase performance without sacrificing reliability, certain type pairs are preloaded into the cache.  In this case, there will be no difference in results between calling `rebuild=True` and `rebuild=False`, but calling with `rebuild=True` will be noticeably slower.

Certain pairs of types are preloaded into ROBOKOP's cache, so there is no point in using rebuild for them. The following list will be updated as the preloaded list is modified.  Note that with the data loaded, it doesn't matter which type is the query and which is the resut.  That is, if a row in this table specifies `disease` and `phenotypic_feature`, then there is no reason to use rebuild for `type1='disease' type2='phenotypic_feature'` or `type1='phenotypic_feature' type2='disease'`.

See the "Expand" notebook for the most up-to-date list of features.

## p-values and numtype1

The calculation of p-value in an enrichment requires the total number of things of type 1 that exist.  So if type 1 is genes, then the p-value requires the total number of genes.  This value is passed in the (optional) parameter `numtype1`.  If numtype1 is not specified, then the ROBOKOP enrichment service will estimate it from the cache.  If the entity type is one that has been cached (_i.e._ it's in the table above), then numtype should be well specified.  If not, then passing in an estimate will give a more robust p-value.

The overall effect of changing numtype1 is not to change the order of the results, but the overall scaling.  So if the caller is not concerned about the correct statistical significance of the p-value, it may be more useful to ignore numtype1 and use maxresults to control which results are used.

Let's see how changing numtype1 changes the results.  Here's the top 10 genes enriched for Fanconi Anemia.

In [8]:
fa='MONDO:0019391'
values = enrichment('disease',[fa],'gene',threshhold=0.2,maxresults=10,rebuild=True)
pd.DataFrame(values)

Return Status: 200


,id,name,p
0,HGNC:28748,SLX1B,0.000342
1,HGNC:20922,SLX1A,0.000342
2,HGNC:28623,FANCD2OS,0.000342
3,HGNC:10071,RNF8,0.000683
4,HGNC:30298,UIMC1,0.000683
5,HGNC:13620,FBH1,0.000683
6,HGNC:17660,DCLRE1A,0.000683
7,HGNC:29814,MUS81,0.000797
8,HGNC:33499,ATRIP,0.000911
9,HGNC:20994,ZSCAN2,0.000911


In this case, the number of diseases is being estimated from the database.  We could enforce a particular number, say 20000, like this, and it will affect the p-values, but not the order of the results:

In [9]:
values = enrichment('disease',[fa],'gene',threshhold=0.2,maxresults=10,rebuild=True,numtype1=20000)
pd.DataFrame(values)

Return Status: 200


,id,name,p
0,HGNC:28748,SLX1B,0.00015
1,HGNC:20922,SLX1A,0.00015
2,HGNC:28623,FANCD2OS,0.00015
3,HGNC:10071,RNF8,0.00030
4,HGNC:30298,UIMC1,0.00030
5,HGNC:13620,FBH1,0.00030
6,HGNC:17660,DCLRE1A,0.00030
7,HGNC:29814,MUS81,0.00035
8,HGNC:33499,ATRIP,0.00040
9,HGNC:20994,ZSCAN2,0.00040


## Sharpening enrichment with ontological descendants

The enrichment service is capable of extending the input identifiers by including their descendants.  For instance, if the `include_descendants` parameter is True and a single fanconi anemia term is specified as the input, the service will automatically consult MONDO to find all subtypes of diabetes and include them as input in the original calculation.

We just looked at the genes enriched for Fanconi Anemia alone.   If we set include_descendants, we will be enriching on FA, and also on anything that descends from FA in the MONDO ontology.

In [10]:
values = enrichment('disease',[fa],'gene',threshhold=0.05,maxresults=0,rebuild=True,include_descendants=True)
pd.DataFrame(values)

Return Status: 200


,id,name,p
0,HGNC:3583,FANCB,0.000003
1,HGNC:3584,FANCC,0.000007
2,HGNC:3586,FANCE,0.000010
3,HGNC:3588,FANCG,0.000010
4,HGNC:28623,FANCD2OS,0.000015
5,HGNC:3587,FANCF,0.000017
6,HGNC:3585,FANCD2,0.000095
7,HGNC:6764,MAD2L2,0.000321
8,HGNC:23845,SLX4,0.000321
9,HGNC:9820,RAD51C,0.000441


## Identifier prefixes and synonymization

The enrichment service will make an effort to cross-interpret different identifiers.  For instance, Fanconi Anemia has an identifer from MONDO, but it also has an identifier from Disease Ontology (DOID:13636).   The results returned from the enrichment service should be independent of the identifier used:

In [11]:
values = enrichment('disease',['DOID:13636'],'gene',threshhold=0.05,maxresults=0,include_descendants=True)
pd.DataFrame(values)

Return Status: 200


,id,name,p
0,HGNC:3583,FANCB,0.000003
1,HGNC:3584,FANCC,0.000007
2,HGNC:3586,FANCE,0.000010
3,HGNC:3588,FANCG,0.000010
4,HGNC:28623,FANCD2OS,0.000015
5,HGNC:3587,FANCF,0.000017
6,HGNC:3585,FANCD2,0.000095
7,HGNC:6764,MAD2L2,0.000321
8,HGNC:23845,SLX4,0.000321
9,HGNC:9820,RAD51C,0.000441
